In [2]:
library(dplyr)
library(tidyverse)
library(moments)
library(rstatix)




Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.1     ✔ readr     2.1.5
✔ ggplot2   4.0.0     ✔ stringr   1.5.2
✔ lubridate 1.9.4     ✔ tibble    3.3.0
✔ purrr     1.1.0     ✔ tidyr     1.3.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘rstatix’


The following object is masked from ‘package:stats’:

    filter




In [4]:
data <- readxl::read_excel("data_hraci.xlsx", sheet = "hraci", skip = 0)
head(data)

IDhrace,odehrane_hod_2018,odehrane_hod_2019,system
<dbl>,<dbl>,<dbl>,<chr>
1,209.9,373.1,OSX
2,269.5,280.3,OSX
3,159.3,290.0,OSX
4,320.0,353.7,OSX
5,240.4,400.0,OSX
6,220.6,222.3,OSX


In [5]:
df_long <- data %>%
  pivot_longer(
    cols = c(odehrane_hod_2018, odehrane_hod_2019), # 1. Které sloupce chceme "splácnout"
    names_to = "Rok",                               # 2. Jak se bude jmenovat nový sloupec s roky
    values_to = "Hodiny",                           # 3. Jak se bude jmenovat nový sloupec s čísly (hodnotami)
    names_prefix = "odehrane_hod_"                  # 4. MAGIE: Tímhle řekneš Rku, ať ten text "odehrane_hod_" ignoruje a smaže ho!
  )
df_long

IDhrace,system,Rok,Hodiny
<dbl>,<chr>,<chr>,<dbl>
1,OSX,2018,209.9
1,OSX,2019,373.1
2,OSX,2018,269.5
2,OSX,2019,280.3
3,OSX,2018,159.3
3,OSX,2019,290.0
4,OSX,2018,320.0
4,OSX,2019,353.7
5,OSX,2018,240.4


In [13]:
# 1. Filtrace: Windows a rok 2018
# Použijeme původní 'data' (ne df_long), protože tam ten sloupec existuje rovnou
data_win <- data %>%
  filter(system == "WIN") %>%
  select(IDhrace, odehrane_hod_2018) 

# 2. Výpočet hranic s parametrem na.rm = TRUE (TOHLE JE TA OPRAVA 🔧)
Q1 <- quantile(data_win$odehrane_hod_2018, 0.25, na.rm = TRUE)
Q3 <- quantile(data_win$odehrane_hod_2018, 0.75, na.rm = TRUE)
IQR_hodnota <- IQR(data_win$odehrane_hod_2018, na.rm = TRUE)

# Hradby
DM <- Q1 - 1.5 * IQR_hodnota
HM <- Q3 + 1.5 * IQR_hodnota

print(paste("Dolní mez:", DM))
print(paste("Horní mez:", HM))

# 3. Identifikace odlehlých pozorování
# I tady musíme ošetřit NA, jinak filter spadne (přidáme !is.na)
odlehle <- data_win %>%
  filter(!is.na(odehrane_hod_2018)) %>% 
  filter(odehrane_hod_2018 < DM | odehrane_hod_2018 > HM)

print("Odlehlá pozorování:")
print(odlehle)

# 4. Nový kvartil (bez odlehlých)
data_clean <- data_win %>%
  filter(!is.na(odehrane_hod_2018)) %>%
  filter(odehrane_hod_2018 >= DM & odehrane_hod_2018 <= HM)

novy_Q3 <- quantile(data_clean$odehrane_hod_2018, 0.75, na.rm = TRUE)

print(paste("Nový horní kvartil:", novy_Q3))

vysledek <- quantile(data_clean$odehrane_hod_2018, 0.75, na.rm = TRUE) # Nový horní kvartil
print(paste("Výsledek z čistých dat:", round(vysledek, 2)))

[1] "Dolní mez: 184.5125"
[1] "Horní mez: 414.8125"
[1] "Odlehlá pozorování:"
# A tibble: 2 × 2
  IDhrace odehrane_hod_2018
    <dbl>             <dbl>
1      94              160.
2      95              155.
[1] "Nový horní kvartil: 328.825"
[1] "Výsledek z čistých dat: 328.82"
